In [ ]:
import numpy as np
import time
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tensorflow as tf
import torchvision
import torch

from sklearn.metrics import accuracy_score

from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable

# IMPORTING FUNCTIONS

In [ ]:
from CNN_PGD import *

# IMPORTING DATA

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(),
)

testing_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

In [ ]:
BATCH_SIZE = 100

data_loaders = {
    'train' : DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1),
    'test'  : DataLoader(testing_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1),
}

# FIRST CLASSIFIER

In [ ]:
# PARAMETERS

# CNN Training Params
EPOCHS = 10
LEARNING_RATE = 0.001

# PGD Params
eps = 0.3
alpha = 0.02
iterations = 20

In [ ]:
classifier_one = CNN()
optimizer_one = optim.Adam(classifier_one.parameters(), lr = LEARNING_RATE)  

In [ ]:
# TRAINING

(one_orginal_images, one_pgd_images, one_pgd_labels, 
 one_training_loss, one_training_accuracy, 
 one_adv_training_loss, one_adv_training_accuracy) = pgd_train(cnn_model = classifier_one, 
                                                               data_load = data_loaders, 
                                                               optim = optimizer_one, 
                                                               pgd_function = untargeted_pgd_attack, 
                                                               EPOCHS = EPOCHS, 
                                                               eps = eps, 
                                                               alpha = alpha,
                                                               iterations = iterations)

In [ ]:
# TESTING

(one_test_accuracy, one_adv_test_accuracy) = pgd_test(pgd_test(cnn_model = classifier_one, 
                                                      data_load = data_loaders, 
                                                      pgd_function = untargeted_pgd_attack, 
                                                      eps = eps, 
                                                      alpha = alpha,
                                                      iterations = iterations,
                                                      details=True)

# SECOND CLASSIFIER

In [ ]:
# PARAMETERS

# CNN Training Params
EPOCHS = 2
LEARNING_RATE = 0.001

# PGD Params
eps = 0.3
alpha = 0.5
iterations = 1

In [ ]:
classifier_two = CNN()
optimizer_two = optim.Adam(classifier_two.parameters(), lr = LEARNING_RATE) 

In [ ]:
# TRAINING

(two_orginal_images, two_pgd_images, two_pgd_labels, 
 two_training_loss, two_training_accuracy, 
 two_adv_training_loss, two_adv_training_accuracy) = pgd_train(cnn_model = classifier_two, 
                                                               data_load = data_loaders, 
                                                               optim = optimizer_two, 
                                                               pgd_function = untargeted_pgd_attack, 
                                                               EPOCHS = EPOCHS, 
                                                               eps = eps, 
                                                               alpha = alpha,
                                                               iterations = iterations)

In [ ]:
# TESTING

(two_test_accuracy, two_adv_test_accuracy) = pgd_test(cnn_model = classifier_two, 
                                                      data_load = data_loaders, 
                                                      pgd_function = untargeted_pgd_attack, 
                                                      eps = eps, 
                                                      alpha = alpha,
                                                      iterations = iterations,
                                                      details=True)

In [ ]:
# GET A FEW IMAGES (UNPERTURBED, PERTURBED FROM TRAINING SET)

# THIRD CLASSIFIER

In [ ]:
# PARAMETERS

# CNN Training Params
EPOCHS = 10
LEARNING_RATE = 0.001

# PGD Params
eps = 0.3
alpha = 0.02
iterations = 20

In [ ]:
classifier_three = CNN()
optimizer_three = optim.Adam(classifier_three.parameters(), lr = LEARNING_RATE) 

In [ ]:
# TRAINING

(three_orginal_images, three_pgd_images, three_pgd_labels, 
 three_training_loss, three_training_accuracy, 
 three_adv_training_loss, three_adv_training_accuracy) = pgd_train(cnn_model = classifier_three, 
                                                               data_load = data_loaders, 
                                                               optim = optimizer_three, 
                                                               pgd_function = targeted_pgd_attack, 
                                                               EPOCHS = EPOCHS, 
                                                               eps = eps, 
                                                               alpha = alpha,
                                                               iterations = iterations)

In [ ]:
# TESTING

(three_test_accuracy, three_adv_test_accuracy) = pgd_test(cnn_model = classifier_three, 
                                                      data_load = data_loaders, 
                                                      pgd_function = targeted_pgd_attack, 
                                                      eps = eps, 
                                                      alpha = alpha,
                                                      iterations = iterations,
                                                      details=True)